In [230]:
import pandas as pd
import random
import datetime

In [8]:
items = pd.read_csv("./item.csv")
items = items.set_index("ID")
items.head()

,name,customer_price,restock_price,customer_amount,restock_amount,order_unit,inventory,Type
ID,,,,,,,,
1,Steak,8.29,25.0,3.00,30,pounds,135,Protein
2,Ground Beef,8.19,54.6,1.00,10,pounds,56,Protein
3,Chicken,7.29,25.0,2.75,30,pounds,97,Protein
4,Vegetables,7.29,15.0,3.50,30,pounds,145,Protein
5,Taco Shells,0.00,20.0,2.00,100,shells,245,Entree Base


In [11]:
customer_orders = pd.DataFrame(columns=["id", "price", "time_of_order", "employee_id"])
customer_order_items = pd.DataFrame(columns=["id", "name", "price"])
co_to_coi = pd.DataFrame(columns=["co_id", "coi_id"])
coi_to_i = pd.DataFrame(columns=["coi_id", "i_id"])

In [17]:
random.choice(list(items[items["Type"] == "Protein"].index))

2

In [133]:
def generate_entree():      # generate entree and return item ids
    ing_ids = []    # list of ids for ingredients
    
    # generate base
    ing_ids.append(random.choice(list(items[items["Type"] == "Entree Base"].index)))
    
    # generate protein
    ing_ids.append(random.choice(list(items[items["Type"] == "Rice"].index)))
    ing_ids.append(random.choice(list(items[items["Type"] == "Beans"].index)))
    ing_ids.append(random.choice(list(items[items["Type"] == "Protein"].index)))
    
    # add cheese
    prob = random.random()
    if prob < 0.4:
        ing_ids.append(list(items[items["name"] == "Cheddar Cheese"].index)[0])
    elif prob < 0.8:
        ing_ids.append(list(items[items["name"] == "American Cheese"].index)[0])
    
    # generate list of toppings
    toppings = items[items["Type"] == "Toppings"]
    num_toppings = random.randint(0, toppings.shape[0])
    topping_ids = random.sample(list(toppings.index), num_toppings)
    topping_ids.sort()
    ing_ids.extend(topping_ids)
    
    # add sauce
    prob = random.random()
    if prob < 0.85:     # one sauce
        ing_ids.append(random.choice(list(items[items["Type"] == "Sauces"].index)))
    elif prob < 0.95:   # two sauces
        sauces = items[items["Type"] == "Sauces"]
        sauce_ids = random.sample(list(sauces.index), 2)
        sauce_ids.sort()
        ing_ids.extend(sauce_ids)
    
    return ing_ids

def generate_side():
    return random.choice(list(items[items["Type"] == "Sides"].index))

def generate_drink():
    return random.choice(list(items[items["Type"] == "Drinks"].index))

In [212]:
def generate_order():
    coi_ids = []
    
    global customer_order_items
    global coi_to_i
    
    entree_ids = generate_entree()
    entree_price = 0
    entree_protein = ""
    entree_base = ""
    for ind in entree_ids:
        item_type = items.loc[ind]["Type"]
        if entree_protein == "" and item_type == "Protein":
            entree_protein = items.loc[ind]["name"]
        elif entree_base == "" and item_type == "Entree Base":
            item_name = items.loc[ind]["name"]
            if (item_name == "Bowls"):
                entree_base = "Bowl"
            elif (item_name == "Taco Shells"):
                entree_base = "Tacos"
            elif (item_name == "Burrito Tortilla"):
                entree_base = "Burrito"
            else:
                entree_base = "Salad"
        entree_price += items.loc[ind]["customer_price"]
    entree_name = entree_protein + " " + entree_base
    coi_id = customer_order_items.shape[0] + 1
    
    customer_order_items = customer_order_items.append({
            "id": coi_id,
            "name": entree_name,
            "price": entree_price
        }, ignore_index=True)
    
    for ent_id in entree_ids:
        coi_to_i = coi_to_i.append({
                "coi_id": coi_id,
                "i_id": ent_id
            }, ignore_index=True)
        
    coi_ids.append(coi_id)
        
    prob_side = random.random()
    if prob_side < 0.2:
        side_id = generate_side()
        side_price = items.loc[side_id]["customer_price"]
        side_name = items.loc[side_id]["name"]
        coi_id = customer_order_items.shape[0] + 1
        
        customer_order_items = customer_order_items.append({
                "id": coi_id,
                "name": side_name,
                "price": side_price
            }, ignore_index=True)
        
        coi_to_i = coi_to_i.append({
                "coi_id": coi_id,
                "i_id": side_id
            }, ignore_index=True)
        
        coi_ids.append(coi_id)
        
    prob_drink = random.random()
    if prob_side < 0.6:
        drink_id = generate_drink()
        drink_price = items.loc[drink_id]["customer_price"]
        drink_name = items.loc[drink_id]["name"]
        coi_id = customer_order_items.shape[0] + 1
        
        customer_order_items = customer_order_items.append({
                "id": coi_id,
                "name": drink_name,
                "price": drink_price
            }, ignore_index=True)
        
        coi_to_i = coi_to_i.append({
                "coi_id": coi_id,
                "i_id": drink_id
            }, ignore_index=True)
        
        coi_ids.append(coi_id)
        
    return coi_ids

In [286]:
def process_customer_order(time_of_order, employee_id):
    global customer_orders
    global co_to_coi
    
    coi_ids = generate_order()
    co_id = customer_orders.shape[0] + 1
    total = 0
    for coi_id in coi_ids:
        total += customer_order_items[customer_order_items["id"] == coi_id]["price"].iloc[0]
        
    customer_orders = customer_orders.append({
            "id": co_id,
            "price": total,
            "time_of_order": time_of_order.strftime("%Y-%m-%d %H:%M:%S"),
            "employee_id": employee_id
        }, ignore_index=True)
    
    for coi_id in coi_ids:
        co_to_coi = co_to_coi.append({
                "co_id": co_id,
                "coi_id": coi_id
            }, ignore_index=True)
    


In [305]:
# clear all tables
customer_orders = customer_orders.iloc[0:0]
customer_order_items = customer_order_items.iloc[0:0]
coi_to_i = coi_to_i.iloc[0:0]
co_to_coi = co_to_coi.iloc[0:0]

In [306]:
open_time = datetime.datetime.strptime('2022-09-12 10:00:00', "%Y-%m-%d %H:%M:%S")
close_time = datetime.datetime.strptime('2022-09-12 16:00:00', "%Y-%m-%d %H:%M:%S")
current_dt = open_time
cashier = random.randint(1, 8)

for i in range(5):
    min_delta = 1.0
    min_scale = 0.1
    if datetime.date.weekday(open_time) == 4:
        min_delta = 0.5
    while (current_dt < close_time):
        current_dt += datetime.timedelta(minutes = (random.gauss(0, 1) * min_scale) + min_delta)
        process_customer_order(current_dt, cashier)
    open_time += datetime.timedelta(days=1)
    close_time += datetime.timedelta(days=1)
    current_dt = open_time
    cashier = random.randint(1, 8)

In [308]:
open_time = datetime.datetime.strptime('2022-09-19 10:00:00', "%Y-%m-%d %H:%M:%S")
close_time = datetime.datetime.strptime('2022-09-19 16:00:00', "%Y-%m-%d %H:%M:%S")
current_dt = open_time
cashier = random.randint(1, 8)

for i in range(5):
    min_delta = 1.0
    min_scale = 0.1
    if datetime.date.weekday(open_time) == 4:
        min_delta = 0.5
    while (current_dt < close_time):
        current_dt += datetime.timedelta(minutes = (random.gauss(0, 1) * min_scale) + min_delta)
        process_customer_order(current_dt, cashier)
    open_time += datetime.timedelta(days=1)
    close_time += datetime.timedelta(days=1)
    current_dt = open_time
    cashier = random.randint(1, 8)

In [309]:
open_time = datetime.datetime.strptime('2022-09-26 10:00:00', "%Y-%m-%d %H:%M:%S")
close_time = datetime.datetime.strptime('2022-09-26 16:00:00', "%Y-%m-%d %H:%M:%S")
current_dt = open_time
cashier = random.randint(1, 8)

for i in range(5):
    min_delta = 1.0
    min_scale = 0.1
    while (current_dt < close_time):
        current_dt += datetime.timedelta(minutes = (random.gauss(0, 1) * min_scale) + min_delta)
        process_customer_order(current_dt, cashier)
    open_time += datetime.timedelta(days=1)
    close_time += datetime.timedelta(days=1)
    current_dt = open_time
    cashier = random.randint(1, 8)

In [310]:
customer_orders.sum()

id                                                        18754750
price                                                     57799.65
time_of_order    2022-09-12 10:01:062022-09-12 10:02:052022-09-...
employee_id                                                  23773
dtype: object

In [297]:
datetime.date.weekday(datetime.datetime.strptime('2022-09-16 10:00:00', "%Y-%m-%d %H:%M:%S"))

4

In [329]:
coi_to_i.to_csv("./csv-files/coi_to_i.csv", index=False)
co_to_coi.to_csv("./csv-files/co_to_coi.csv", index=False)
customer_order_items.to_csv("./csv-files/customer_order_items.csv", index=False)
customer_orders.to_csv("./csv-files/customer_orders.csv", index=False)

In [330]:
customer_orders.columns

Index(['id', 'price', 'time_of_order', 'employee_id', 'date', 'time'], dtype='object')

In [331]:
customer_orders["date"]  = customer_orders["time_of_order"].str[:10]

In [332]:
customer_orders.groupby("date").sum()

,price
date,
2022-09-12,3420.99
2022-09-13,3393.37
2022-09-14,3395.37
2022-09-15,3427.65
2022-09-16,6837.89
2022-09-19,3451.78
2022-09-20,3307.32
2022-09-21,3424.14
2022-09-22,3425.38


In [333]:
customer_orders["time"] = customer_orders["time_of_order"].str[11:]

In [334]:
customer_orders["time"].min()

'10:00:25'

In [335]:
customer_orders.tail()

,id,price,time_of_order,employee_id,date,time
6119,6120,10.38,2022-09-30 15:56:30,6,2022-09-30,15:56:30
6120,6121,7.29,2022-09-30 15:57:25,6,2022-09-30,15:57:25
6121,6122,9.58,2022-09-30 15:58:32,6,2022-09-30,15:58:32
6122,6123,9.29,2022-09-30 15:59:31,6,2022-09-30,15:59:31
6123,6124,10.38,2022-09-30 16:00:34,6,2022-09-30,16:00:34
